# Installations

In [ ]:
!pip install git+https://github.com/facebookresearch/myosuite.git
!pip install tqdm
%env MUJOCO_GL=egl

# Imports

In [ ]:
from myosuite.simhive.myo_sim.test_sims import TestSims as loader
import scipy.optimize as optimize
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
import numpy as np
import skvideo.io
import mujoco
import mujoco.viewer
import csv
import sys
import os

# Introduction and core function
In this tutorial a target trajectory will be replicated by MyoHand using MuJoCo inverse dynamics, i.e., given a sequence of joint angles *qpos*, a sequence of muscle activations *act* will be generated.
To achieve this goal, the following optimization problem will be solved at each time step:

$$
\begin{align*}
\min_{act}~&\|~moment~\cdot~(gain~*~act~+~bias)~-~qfrc~\| \\
s.t.~~& 0 \leq~act~\leq~1 \\
& act_{t}~-~act_{t-1}~\leq~0.1
\end{align*}
$$

The last constraint is to prevent *act* to change too much w.r.t. the previous state.

The resulting muscle activations must be set directly on the actuators, without going through *ctrl*, since muscle filter dynamics is not taken into account in the optimization problem (see https://mujoco.readthedocs.io/en/stable/modeling.html#muscles). Therefore, in the next section, actuator dynamics is initialized as simple integrator (instead of muscle filter).

In [ ]:
def get_act(model, data, target_qpos):
    """
    Compute the muscle activations needed to reach the target position in the next mujoco step.
    """
    data_copy = deepcopy(data)
    # ---- qfrc computation
    data_copy.qpos = target_qpos.copy()
    data_copy.qvel = (target_qpos - data.qpos)/model.opt.timestep
    mujoco.mj_forward(model, data_copy)
    data_copy.qacc = 0
    mujoco.mj_inverse(model, data_copy)
    qfrc = data_copy.qfrc_inverse.copy()
    # ---- act computation
    gain = np.array([])
    bias = np.array([])
    bounds = []
    bounds_margin = 0.10
    for idx_actuator in range(model.nu):
        length = data_copy.actuator_length[idx_actuator]
        lengthrange = model.actuator_lengthrange[idx_actuator]
        velocity = data_copy.actuator_velocity[idx_actuator]
        acc0 = model.actuator_acc0[idx_actuator]
        prmb = model.actuator_biasprm[idx_actuator,:9]
        prmg = model.actuator_gainprm[idx_actuator,:9]
        bias = np.append(bias, mujoco.mju_muscleBias(length, lengthrange, acc0, prmb))
        gain = np.append(gain, mujoco.mju_muscleGain(length, velocity, lengthrange, acc0, prmg))
        bounds.append((max(0, data.act[idx_actuator]-bounds_margin), min(1, data.act[idx_actuator]+bounds_margin)))
    cost_function = lambda act: np.linalg.norm((gain * act + bias) @ data_copy.actuator_moment - qfrc)
    result = optimize.minimize(cost_function, data.act, method='SLSQP', bounds=tuple(bounds))
    return result.x

# Main

In [ ]:
# ---- initializations
model = loader.get_sim(None, 'hand/myohand.xml')
model.actuator_dyntype = mujoco.mjtDyn.mjDYN_INTEGRATOR # instead of mjDYN_MUSCLE
data = mujoco.MjData(model)
data_folder = '6_data/'
input_file = '6_trajectory.csv'
output_file = '6_activations.csv'
traj = pd.read_csv(data_folder+input_file).values
all_act = np.array([mujoco.mj_id2name(model, mujoco.mju_str2Type('actuator'.encode(sys.getdefaultencoding())), i) for i in range(model.nu)])

In [ ]:
# ---- main loop
for idx in tqdm(range(traj.shape[0])):
    target_qpos = traj[idx, 1:]
    act = get_act(model, data, target_qpos)
    all_act = np.vstack((all_act, act))
    data.act = act
    mujoco.mj_step(model, data)

In [ ]:
# ---- output saving
with open(data_folder+output_file, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(all_act)

# Results

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path, video_width = 400):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video autoplay width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
# ---- camera settings
AZIMUTH = 166.553
DISTANCE = 1.178
ELEVATION = -36.793
LOOKAT = [-0.93762553, -0.34088276, 0.85067529]

In [ ]:
# ---- video generation
data_ref = mujoco.MjData(model) # data for reference trajectory
data_ach = mujoco.MjData(model) # data for achieved trajectory
traj = pd.read_csv(data_folder+input_file).values
acts = pd.read_csv(data_folder+output_file).values
output_video = '6_output_video.mp4'

camera = mujoco.MjvCamera()
camera.azimuth = AZIMUTH
camera.distance = DISTANCE
camera.elevation = ELEVATION
camera.lookat = np.array(LOOKAT)
options_ref = mujoco.MjvOption()
options_ref.flags[:] = 0
options_ref.geomgroup[1:] = 0
options_ach = mujoco.MjvOption()
options_ach.flags[:] = 0
options_ach.flags[[4,5]] = 1 # actuator and activation ON
options_ach.geomgroup[1:] = 0
renderer = mujoco.Renderer(model)
renderer.scene.flags[:] = 0

frames = []
for idx in tqdm(range(traj.shape[0])):
    # ---- ref related
    data_ref.qpos = traj[idx, 1:]
    mujoco.mj_step(model, data_ref)
    renderer.update_scene(data_ref, camera=camera, scene_option=options_ref)
    pixels_ref = renderer.render()
    # ---- ach related
    # required activations are directly set without going through ctrl
    data_ach.act = acts[idx, :]
    mujoco.mj_step(model, data_ach)
    renderer.update_scene(data_ach, camera=camera, scene_option=options_ach)
    pixels_ach = renderer.render()
    # ---- merging
    pixels = np.append(pixels_ref, pixels_ach, axis=1)
    frames.append(pixels)

skvideo.io.vwrite(data_folder+output_video, np.asarray(frames),outputdict={"-pix_fmt": "yuv420p"})

Left, reference trajectory. Right, achieved trajectory by means of muscle activations resulting from inverse dynamics.

In [ ]:
show_video(data_folder+output_video)